In [0]:
from pyspark.sql.functions import col,current_timestamp,sum,to_date
from pyspark.sql.window import Window       

spark.sql("create schema if not exists lakeflow_job.silver")

orders_df =  spark.table("lakeflow_job.bronze.orders_bronze")

orders_df = (orders_df.
 select(col("orderId").alias("order_id")
        ,to_date(col("OrderDate")).alias("order_date")
        ,col("CustomerID").alias("customer_id")
        ,col("TotalAmount").cast("double").alias("total_amount")
        ,col("Status").alias("status")))


orders_df.write.format("delta").mode("overwrite").saveAsTable("lakeflow_job.silver.orders_cleaned_silver")

In [0]:
window = Window.partitionBy("order_id").orderBy(col("order_date").desc())

In [0]:
from pyspark.sql.functions import row_number

orders_cleaned = spark.table("lakeflow_job.silver.orders_cleaned_silver")

orders_cleaned_rn = orders_cleaned.withColumn("rn", row_number().over(window))

In [0]:
total_cleaned = orders_cleaned.count()
total_dedup = orders_cleaned_rn.filter(col("rn")==1).count()

In [0]:
print(total_cleaned)
print(total_dedup)

In [0]:
duplicates_exist = total_cleaned != total_dedup

In [0]:
dbutils.jobs.taskValues.set(key="duplicates_exist", value=duplicates_exist)